In [17]:
import dash
from dash import dcc, html, Input, Output
import dash_bootstrap_components as dbc
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.ensemble import RandomForestClassifier


In [18]:
# Load data
df = pd.read_csv("datasets/watering plants.csv")
# Initialize Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])



In [19]:


# Define app layout
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col(html.H1("Plant Watering System Dashboard", className="text-center my-4"), width=12)
    ]),
    
    dbc.Row([
        dbc.Col([
            html.H3("Data Overview", className="text-center"),
            dcc.Markdown(f"""
                *Dataset Stats:*
                - Total records: {len(df)}
                - Pump ON: {df['Pump Data'].sum()} ({df['Pump Data'].mean()*100:.1f}%)
                - Pump OFF: {len(df) - df['Pump Data'].sum()}
            """)
        ], width=3),
        
        dbc.Col([
            dcc.Graph(id='pump-distribution',
                     figure=px.pie(df, names='Pump Data', title='Pump Activation Distribution'))
        ], width=3),
        
        dbc.Col([
            dcc.Graph(id='soil-hist',
                     figure=px.histogram(df, x='Soil Moisture', color='Pump Data',
                                        title='Soil Moisture Distribution',
                                        barmode='overlay'))
        ], width=6)
    ]),
    
    dbc.Row([
        dbc.Col([
            dcc.Graph(id='temp-humidity-scatter',
                     figure=px.scatter(df, x='Temperature', y='Air Humidity',
                                      color='Pump Data', title='Temp vs Humidity'))
        ], width=6),
        
        dbc.Col([
            dcc.Graph(id='soil-pump-scatter',
                     figure=px.scatter(df, x='Soil Moisture', y='Pump Data',
                                      title='Soil Moisture vs Pump Status',
                                      trendline="lowess"))
        ], width=6)
    ]),
    
    dbc.Row([
        dbc.Col([
            html.H3("Feature Correlation"),
            dcc.Graph(id='corr-heatmap',
                     figure=px.imshow(df.corr(), text_auto=True,
                                     title='Feature Correlation Matrix'))
        ], width=6),
        
        dbc.Col([
            html.H3("3D Relationship"),
            dcc.Graph(id='3d-plot',
                     figure=px.scatter_3d(df, x='Soil Moisture', y='Temperature',
                                         z='Air Humidity', color='Pump Data',
                                         title='3D View of Parameters'))
        ], width=6)
    ]),
    
    dbc.Row([
        dbc.Col([
            html.H3("Threshold Analysis"),
            dcc.Slider(
                id='soil-threshold',
                min=300,
                max=1000,
                step=10,
                value=650,
                marks={i: str(i) for i in range(300, 1001, 100)}
            ),
            dcc.Graph(id='threshold-plot')
        ], width=12)
    ])
], fluid=True)

# Callback for threshold analysis
@app.callback(
    Output('threshold-plot', 'figure'),
    Input('soil-threshold', 'value')
)
def update_threshold_plot(threshold):
    df['Predicted'] = (df['Soil Moisture'] < threshold).astype(int)
    accuracy = (df['Predicted'] == df['Pump Data']).mean()
    
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=df['Soil Moisture'],
        y=df['Pump Data'],
        mode='markers',
        name='Actual',
        marker=dict(color='blue', opacity=0.5)
    ))
    
    fig.add_trace(go.Scatter(
        x=df['Soil Moisture'],
        y=df['Predicted'],
        mode='markers',
        name='Predicted',
        marker=dict(color='red', opacity=0.5)
    ))
    
    fig.add_vline(x=threshold, line_dash="dash", line_color="green",
                 annotation_text=f"Threshold: {threshold}", 
                 annotation_position="top right")
    
    fig.update_layout(
        title=f"Pump Activation Threshold (Accuracy: {accuracy:.2%})",
        xaxis_title="Soil Moisture",
        yaxis_title="Pump Status",
        showlegend=True
    )
    
    return fig

# Run the app
if __name__ == '__main__':
    app.run(debug=True)
